Import Libraries/Data

In [1]:
import os
import sys
import glob
import numpy as np
import torch
from torchvision import transforms, datasets
from PIL import Image

#Import image loaders
os.chdir("../../src/")
from load_data import preview_images, load_images

In [2]:
#Import and preview image data
train_path= "../Data/train/"
horse_train= load_images(train_path + "horses/")
human_train= load_images(train_path + "humans/")

val_path= "../Data/validation/"
horse_test= load_images(val_path + "horses/")
human_test= load_images(val_path + "humans/")

#Sanity Check
print("Training Horse Data: %s \nTest Horse Data: %s\n" % (horse_train.shape, horse_test.shape))
print("Training Human Data: %s \nTest Human Data: %s" % (human_train.shape, human_test.shape))

Training Horse Data: (500, 300, 300, 4) 
Test Horse Data: (128, 300, 300, 4)

Training Human Data: (527, 300, 300, 4) 
Test Human Data: (128, 300, 300, 4)


Preprocess **Training** Data

In [3]:
#Assign labels: 1= human, 0 = horse
y_train_human= np.full((len(human_train),1), 1)
y_train_horse= np.full((len(horse_train),1), 0)

#Concatenate Training data
X= np.concatenate((horse_train, human_train), axis=0)
y= np.concatenate((y_train_horse, y_train_human), axis= 0)

#Zero-center data
training_mean= X.mean()
training_std= X.std()
X= (X - training_mean)/training_std

#Shuffle, reshape and convert to tensors
from sklearn.utils import shuffle
X_train, y_train= shuffle(X, y)

X_train= X_train.reshape((-1, 4, 300, 300))

X_train= torch.from_numpy(X_train)
y_train= torch.from_numpy(y_train)

#Sanity Check
print(X_train.shape)
print(y_train.shape)
#Wrap tensors into Dataset 
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)    

torch.Size([1027, 4, 300, 300])
torch.Size([1027, 1])


In [4]:
#Preprocess Test Data
y_test_human= np.full((len(human_test),1), 1)
y_test_horse= np.full((len(horse_test),1), 0)

#Concatenate validation set
X= np.concatenate((horse_test, human_test), axis=0)
y= np.concatenate((y_test_horse, y_test_human), axis= 0)

#Zero-Center data
X= (X- training_mean)/training_std

#Shuffle, reshape, and convert test data to tensors
X_test, y_test= shuffle(X, y)

X_test= X_test.reshape((-1, 4, 300, 300))

X_test= torch.from_numpy(X_test)
y_test= torch.from_numpy(y_test)

#Sanity Check
print("Test data shape: %s \nTest label shape: %s" % (X_test.shape, y_test.shape))

#Wrap tensors into Dataset 
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

Test data shape: torch.Size([256, 4, 300, 300]) 
Test label shape: torch.Size([256, 1])


# Modeling

Base ANN 

In [5]:
#Use base model from early computer vision
from models import BaseNeuralNetwork, train_loop

device= "cpu"
base_model= BaseNeuralNetwork().to(device)
print(base_model)

BaseNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=360000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


Base ANN training

In [6]:
from torch import optim
from torch import nn
from torch.utils.data import DataLoader


#Set up loss function, optimizer and batch
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(base_model.parameters(),
                      lr= 10 ** -3,
                       momentum= .9)

batch_size= 64

#Create DataLoader iterable
train_dataloader= DataLoader(train_dataset, batch_size= batch_size, shuffle= False)
test_dataloader= DataLoader(test_dataset, batch_size= len(test_dataset), shuffle= False)

#Sanity Check
for x, y in train_dataloader:
    print(x.shape)
    print(y.shape)
    break          
#Sanity Check
for x, y in test_dataloader:
    print(x.shape)
    print(y.shape)
    break 

torch.Size([64, 4, 300, 300])
torch.Size([64, 1])
torch.Size([256, 4, 300, 300])
torch.Size([256, 1])


In [7]:
train_loop(train_dataloader, base_model, criterion, optimizer)

loss: 0.692595  [   64/ 1027]
loss: 0.657384  [  128/ 1027]
loss: 0.596722  [  192/ 1027]
loss: 0.535421  [  256/ 1027]
loss: 0.566393  [  320/ 1027]
loss: 0.510413  [  384/ 1027]
loss: 0.474450  [  448/ 1027]
loss: 0.380675  [  512/ 1027]
loss: 0.411957  [  576/ 1027]
loss: 0.337791  [  640/ 1027]
loss: 0.415704  [  704/ 1027]
loss: 0.377260  [  768/ 1027]
loss: 0.391146  [  832/ 1027]
loss: 0.261502  [  896/ 1027]
loss: 0.293247  [  960/ 1027]
loss: 0.287931  [ 1024/ 1027]
loss: 0.344794  [   51/ 1027]


In [8]:
stop

NameError: name 'stop' is not defined

In [ ]:
X.shape[0]

In [ ]:
for batch, (X, y) in enumerate(train_dataloader):
    print(batch )

In [ ]:
#For Loop
epochs = 5

base_model.train()
for e in range(epochs):
    print("-----------------------")
    print(f"Epoch {e + 1} Results\n-----------------------")
    
    #Training Loop
    size= len(train_dataloader.dataset)
        #compute predictions and loss
    for batch, (X,y) in enumerate(train_dataloader):
        
        #Make predictions and compute loss
        pred= base_model(X.float())
        loss= criterion(pred, y.flatten())
        
        #Always zero gradients for each batch
        optimizer.zero_grad() #Should i move this?
        
        #Calculate gradient        
        loss.backward()
        
        #Adjust model's weights
        optimizer.step()
        
        #Output stats
        if batch % 64 != 0:
            loss, curr = loss.item(), batch * len(X)
            print(f"Loss: {loss:7f} [{curr:>5d}/{size:>5d}]")  
    
    #Test Loop
    size = len(test_dataloader.dataset)
    num_batches= len(test_dataloader)
    test_loss, correct = 0,0
    
    with torch.no_grad(): #turns off gradient calculation
        for X, y in test_dataloader:
            pred= base_model(X.float())
            test_loss += criterion(pred, y.flatten()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Accuracy:\nAccuracy: {(100*correct)/size:>.01f}%, Avg loss: {test_loss:>5f}\n")
    

In [ ]:
len(x)

In [ ]:
for batch, (X,y) in enumerate(train_dataloader):
    print(batch)
    print(X)
    print(y)
    break


In [ ]:
y.flatten().to(device)